### Insurance Customer Churn Prediction
Customer churn refers to the loss of existing clients or customers. This solution identifies Insurance customers who are more likely to close their account and leave the Insurance Company. During the training stage, the solution automatically conducts feature interaction on the training data and selects a subset of features based on feature importance. It then trains multiple models and identifies the best performing model. This model is then selected for prediction on new data.

### Contents

1. [Set up the environment](#Set-up-the-environment)
1. [Usage Instructions](#Usage-Instructions)
1. [Upload the data for training](#Upload-the-data-for-training)
1. [Run Training Job](#Run-Training-Job)
1. [Live Inference Endpoint](#Live Inference)
1. [Batch Transform Job](#Batch-Transform-Job)
1. [Output Interpretation](#Output-Interpretation)



<img src="images/Flow_diagram.JPG">

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

### Input format
#### Input:
Name of the file: <b>train.csv</b><br>
This file contains historical churn prediction data for Ecommerce. T<br><br>

</ul>
<li>  Tenure:Tenure of customer in organization</li>
<li> PreferredLoginDevice: Preferred login device of customer</li>

<li> CityTier: City tier</li>
<li> WarehouseToHome: Distance in between warehouse to home of customer</li>
<li> PreferredPaymentMode: Preferred payment method of customer</li>
<li> Gender: Gender of customer</li>
<li> HourSpendOnApp: Number of hours spend on mobile application or website</li>
    
<li> NumberOfDeviceRegistered: Total number of deceives is registered on particular customer</li>
<li> PreferedOrderCat: Preferred order category of customer in last month</li>
<li> SatisfactionScore: Satisfactory score of customer on service</li>
<li> MaritalStatus: Marital status of customer</li>
<li> NumberOfAddress: Total number of added added on particular customer
    
 <li>Complain: Any complaint has been raised in last month</li>
<li> OrderAmountHikeFromlastYear: Percentage increases in order from last year</li>
<li> CouponUsed: Total number of coupon has been used in last month</li>
<li> OrderCount: Total number of orders has been places in last month</li>
<li> DaySinceLastOrder: Day Since last order by customer</li>
    






## Set up the environment
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [23]:
# S3 prefix
prefix = 'churn-insurance'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session
The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [24]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training
When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using classification dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [25]:
data_location= 's3://mphasis-marketplace/churn-prediction-insurance/input/train.csv'

## Create an estimator and fit the model
In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above

Then we use fit() on the estimator to train against the data that we uploaded above.

In [26]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/telecom-pycaret-churn'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 3, 'ml.c4.2xlarge',
                      output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2021-04-28 09:39:39 Starting - Starting the training job...
2021-04-28 09:39:41 Starting - Launching requested ML instancesProfilerReport-1619602779: InProgress
......
2021-04-28 09:41:05 Starting - Preparing the instances for training......
2021-04-28 09:42:00 Downloading - Downloading input data...
2021-04-28 09:42:36 Training - Downloading the training image....Starting the training.
(33908, 17)
IntProgress(value=0, description='Processing: ', max=3)
                                                                    
                                                                    
Initiated  . . . . . . . . . . . . . . . . . .              09:43:13
Status     . . . . . . . . . . . . . . . . . .  Loading Dependencies
                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .                     09:43:13
Status     . . . . . . . . . . . . . .

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.


In [28]:
training_job_name = tree.latest_training_job.name
attached_tree = sage.estimator.Estimator.attach(training_job_name)



2021-04-28 09:47:57 Starting - Preparing the instances for training
2021-04-28 09:47:57 Downloading - Downloading input data
2021-04-28 09:47:57 Training - Training image download completed. Training in progress.
2021-04-28 09:47:57 Uploading - Uploading generated training model
2021-04-28 09:47:57 Completed - Training job completed



### Deploy the model
Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [29]:

from sagemaker.predictor import csv_serializer
predictor = attached_tree.deploy(4, 'ml.m4.xlarge', serializer=csv_serializer,endpoint_name='churn-insurance-pycaret')

-------------!

## Choose some data and use it for a prediction


In [30]:
test_data  = 's3://mphasis-marketplace/churn-prediction-insurance/input/test.csv'

data = pd.read_csv(test_data,encoding='ISO-8859–1',header=None)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
1,0.571051204,0.406842993,0.984523317,0.011016098,-0.569350638,-0.411453106,-0.251940371,0,1,1,0,0,0,0,11,3
2,-1.124080394,-0.1669349,0.503891812,-0.322932109,0.721810522,0.547323125,0.18219753,0,2,1,0,0,0,0,5,1
3,0.476877226,0.145079409,-0.577529074,-0.691828384,-0.246560348,-0.411453106,-0.251940371,0,1,1,0,0,0,0,1,3
4,1.606964958,-0.447419342,1.82562845,-0.983062285,7.177616323,-0.411453106,-0.251940371,1,1,0,0,1,0,0,5,3


In [31]:
predictions = predictor.predict(data.values).decode('utf-8')



The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [32]:
print(predictions)

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,Label,Score
0.571051204,0.406842993,0.9845233170000001,0.011016098,-0.569350638,-0.411453106,-0.251940371,0,1,1,0,0,0,0,11,3,1,0.7098
-1.1240803940000001,-0.1669349,0.503891812,-0.32293210899999997,0.7218105220000001,0.547323125,0.18219753,0,2,1,0,0,0,0,5,1,0,0.9028
0.476877226,0.145079409,-0.577529074,-0.691828384,-0.246560348,-0.411453106,-0.251940371,0,1,1,0,0,0,0,1,3,0,0.9876
1.606964958,-0.44741934200000005,1.82562845,-0.983062285,7.177616323,-0.411453106,-0.251940371,1,1,0,0,1,0,0,5,3,0,0.9997



### Output

Output files contains column predicted Group, which has the predicted class

In [33]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path)

In [34]:
transformer.transform(test_data, content_type='text/csv')
transformer.wait()
print("Batch Transform output saved to " + transformer.output_path)

............................Starting the inference server with 4 workers.
[2021-04-28 10:02:58 +0000] [12] [INFO] Starting gunicorn 20.1.0
[2021-04-28 10:02:58 +0000] [12] [INFO] Listening at: unix:/tmp/gunicorn.sock (12)
[2021-04-28 10:02:58 +0000] [12] [INFO] Using worker: gevent
[2021-04-28 10:02:58 +0000] [16] [INFO] Booting worker with pid: 16
[2021-04-28 10:02:58 +0000] [17] [INFO] Booting worker with pid: 17
[2021-04-28 10:02:58 +0000] [18] [INFO] Booting worker with pid: 18
[2021-04-28 10:02:58 +0000] [19] [INFO] Booting worker with pid: 19
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [28/Apr/2021:10:03:07 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [28/Apr/2021:10:03:07 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [28/Apr/2021:10:03:07 +0000] "POST /invocations HTTP/1.1" 200 671 "-" "Go-http-client/1.1"
2021-04-

#### Inspect the Batch Transform Output in S3

In [35]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], "test.csv")



s3_client = sess.boto_session.client('s3')

response = s3_client.get_object(Bucket = sess.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')
print(response_bytes)

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,Label,Score
0.571051204,0.406842993,0.9845233170000001,0.011016098,-0.569350638,-0.411453106,-0.251940371,0,1,1,0,0,0,0,11,3,1,0.7098
-1.1240803940000001,-0.1669349,0.503891812,-0.32293210899999997,0.7218105220000001,0.547323125,0.18219753,0,2,1,0,0,0,0,5,1,0,0.9028
0.476877226,0.145079409,-0.577529074,-0.691828384,-0.246560348,-0.411453106,-0.251940371,0,1,1,0,0,0,0,1,3,0,0.9876
1.606964958,-0.44741934200000005,1.82562845,-0.983062285,7.177616323,-0.411453106,-0.251940371,1,1,0,0,1,0,0,5,3,0,0.9997



### View Output
Lets read results of above transform job from s3 files and print output

In [36]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/test.csv.out".format(transform_output_folder), '/tmp/test.csv.out')
with open('/tmp/test.csv.out') as f:
    results = f.readlines() 
##print("Transform results: \n{}".format(''.join(results)))
string_final = ''.join(results)

print(string_final)

with open("Output.txt", "w") as text_file:
    text_file.write(string_final)

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,Label,Score
0.571051204,0.406842993,0.9845233170000001,0.011016098,-0.569350638,-0.411453106,-0.251940371,0,1,1,0,0,0,0,11,3,1,0.7098
-1.1240803940000001,-0.1669349,0.503891812,-0.32293210899999997,0.7218105220000001,0.547323125,0.18219753,0,2,1,0,0,0,0,5,1,0,0.9028
0.476877226,0.145079409,-0.577529074,-0.691828384,-0.246560348,-0.411453106,-0.251940371,0,1,1,0,0,0,0,1,3,0,0.9876
1.606964958,-0.44741934200000005,1.82562845,-0.983062285,7.177616323,-0.411453106,-0.251940371,1,1,0,0,1,0,0,5,3,0,0.9997

